Code to convert zone plate pattern in numpy to the time dependent term in parabolic wave equation and store it as hdf5.

In [1]:
%%capture
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pickle,os

In [2]:
def make_zp_from_rings(n,grid_size):
    zp = np.zeros((grid_size,grid_size))
    for i in range(n):
        if i%2 == 1 :
            locs_ = np.load('ring_locs_'+str(i)+'.npy')
            locs_ = tuple((locs_[0],locs_[1]))
            vals_ = np.load('ring_vals_'+str(i)+'.npy')
            zp[locs_] = vals_
    return zp

*make_zp_from_rings* : make a zone plate from the rings which were created earlier.
* *Inputs* : n - number of rings, grid_size 
* *Outputs* : a numpy array containing the zone plate

Switch to directory containing zone plate array and parameters.

In [3]:
pwd = os.getcwd()
os.chdir(pwd+str('/rings'))

Importing zone plate pattern and the parameters associated with it. Plotting the zone plate after creating it from the rings. One needs to save the image with high dpi to clearly see the zone plate. 

In [4]:
parameters = pickle.load(open('parameters.pickle','rb'))

grid_size = parameters['grid_size']
energy    = parameters['energy(in eV)']
delta     = parameters['delta']
beta      = parameters['beta']
wavel     = parameters['wavelength in m']
dim_x = dim_y = grid_size

ZP = make_zp_from_rings(200,int(grid_size))

Switch back to current working directory.

In [5]:
os.chdir(pwd)

Evaluate expression to obtain time dependent term in PWE.

In [6]:
k = 2*np.pi/wavel
ZP = ZP*((-1j*k)/2)*((1-delta-1j*beta)**2-1)

Flatten dataset and store as 2D array with the first column containin the real part and the second part containing the imaginary part.

In [7]:
ZP_ = np.zeros((dim_x,dim_y,2))
ZP_[:,:,0] = np.real(ZP)
ZP_[:,:,1] = np.imag(ZP)

Create a new hdf5 file to store the dataset. Set the attribute complex for PETSc to interpret the dataset as a 1D complex array.

In [8]:
f = h5py.File("ref_index_dmda.h5", "w")
dset = f.create_dataset("ref_index", (dim_x,dim_y,2), dtype='f8')
dset.attrs.__setitem__("complex",1)

Transfer the data. Close the hdf5 file after the transfer.

In [9]:
dset[:,:,:] = ZP_

f.close()